# Big data w genomice 

Zakres zajęć:
* wprowadzenie do środowiska chmury obliczeniowej, omówienie architektury rozwiązań zbudowanych w oparciu o Apache Spark i Kubernetes
* obiektowa pamięć masowa
* zrównoleglony potok przetwarzania
* dostęp/analiza/wizualizacja danych poprzez rozproszone operacje

## Środowisko Google Kubernetes Engine

## Kubernetes

Kubernetes (k8s) to otwarta platforma do koordynacji wysoko dostępnego klastra. 
* Umożliwia  deklaratywną konfigurację, automatyzację wdrażania, skalowanie i autoskalowanie rozwiązań.
* Pozwala uruchamiać aplikacje/narzędzie bez przypisywania ich do konkretnej maszyny. Aplikacje muszą być niezależne od konkretnego serwera: muszą być skonteneryzowane.
* Możliwe do uruchomienia na prywatnym centrum danych, infrastrukturze hybrydowej lub chmurze publicznej.


**Klaster Kubernetes**  
* **Węzeł sterujący (Master node)** koordynuje działanie klastra np. zlecanie uruchomienia aplikacji, utrzymywanie pożądanego stanu aplikacji, skalowanie aplikacji i instalowanie nowych wersji
* **Węzeł roboczy (Worker node)** Na węzłach uruchamiane są aplikacje. Na każdym węźle działa agent zarządzający tym węzłem i komunikujący się z masterem Kubernetes (API). Węzeł zawiera także narzędzia do obsługi kontenerów

**Pod** to grupa jednego/wielu kontenerów wraz ze wspólnymi zasobami (np. dysk). Pod tworzy "wirtualney serwer" i może zawierać różne kontenery aplikacji współdzielące zasoby i kontekst wykonawczy na tym samym węźle.
* Pod jest uruchamiany na węźle roboczym. Węzeł jest maszyną roboczą, fizyczną lub wirtualną, w zależności od klastra. 
* Węzeł może zawierać wiele podów. 
* Kubernetes master automatycznie zleca uruchomienie podów na różnych węzłach w ramach klastra. Automatyczne zlecanie uruchomienia bierze pod uwagę zasoby dostępne na każdym z węzłów.

![](gke.png)

## Apache Spark na GKE

* Apache Spark platforma ogólnego zastosowania, opensource, do przetwarzania duzych zbiorow danych.
* Posiada  API dla języków programowania: Scala, Python i R. 
* Przetwarzanie w Spark przetwarzanie jest wykonywane w większości  wprost w pamięci operacyjnej.
* Przeznaczenie: do uruchamiania  aplikacji i skryptów z wykorzystaniem uczenia maszynowego lub interaktywnych kwerend.
* Spark ten wspiera SQL (typ DataFrames), przetwarzanie strumieniowe oraz przetwarzanie grafów.
* Integracja z lokalną pamięci masową, rozproszonymi lub obiektowymi systemu plików.
* Spark można uruchamić na pojedynczej maszynie na środowisku klastrowym, lub w chmurze. 
* **Spark jest rozszerzalny**. Z tej właściwości będziemy dzisiaj korzystać


## Obiektowa pamięć masowa (Google Cloud Storage)

**Projekt**
* Wszystkie dane przynależą do konkretnego projektu.
* Do projektu mogą mieć dostęp użytkownicy.
* Projekt ma zdefiniowane metody uwierzytelniające, rozliczenia, monitorowanie etc.

**Kubełek (bucket)** to kontener na pliki/obiekty.
* Nazwa Bucketu musi być unikalna w skali całej usługi u wszystkich użytkowników (!)
* Kubełków nie można zagnieżdzać
* W kubełkach możemy tworzyć foldery i tam logicznie grupować pliki.
* Kubełek wraz z zawartością może zostać udostępniony publicznie.
* Kubełkowi nie można zmienić nazwy lub metadanych. Trzeba go usunąć i stworzyć ponownie. 

**Obiekt**
* obiekty przechowywane w kubełkach
* obiekty mają zawartość oraz metadane
* obiekty są niemodyfikowalne

Do operacji na Google Storage można wykorzystać narzędzie `gsutil`:

Operacje na kubełkach
* listowanie kubełków (buckets) - `ls`
* tworzenie nowego kubełka - `mb`
* usuwania kubełka - `rm`
* listowanie zawartości kubełków - `ls`
* udostępnianie - `iam`

Operacje na obiektach
* dodawania pliku do kubełka - `cp`
* kopiowanie między kubełkami - `cp`
* usuniecie z kubełka - `cp`
* pobranie informacji o obiekcie - `stat`

In [1]:
! gsutil ls | grep edugen

gs://bucket-edugen22/
gs://bucket-edugen33/
gs://edugen-common-data2/
gs://edugen-lab-matdaw2/
gs://edugen-lab-tgambin2/
gs://edugen-test100/


In [2]:
! gsutil mb gs://edugen-lab-${USER}2  # stworzenie swojego bucketu

Creating gs://edugen-lab-tgambin2/...
ServiceException: 409 Bucket edugen-lab-tgambin2 already exists.


In [3]:
! gsutil ls -L -b gs://edugen-lab-${USER}2 # listowanie zawartości

gs://edugen-lab-tgambin2/ :
	Storage class:			STANDARD
	Location type:			multi-region
	Location constraint:		US
	Versioning enabled:		None
	Logging configuration:		None
	Website configuration:		None
	CORS configuration: 		None
	Lifecycle configuration:	None
	Requester Pays enabled:		None
	Labels:				None
	Default KMS key:		None
	Time created:			Thu, 12 May 2022 19:07:21 GMT
	Time updated:			Thu, 12 May 2022 19:07:21 GMT
	Metageneration:			1
	Bucket Policy Only enabled:	False
	ACL:				
	  [
	    {
	      "entity": "project-owners-9940977455",
	      "projectTeam": {
	        "projectNumber": "9940977455",
	        "team": "owners"
	      },
	      "role": "OWNER"
	    },
	    {
	      "entity": "project-editors-9940977455",
	      "projectTeam": {
	        "projectNumber": "9940977455",
	        "team": "editors"
	      },
	      "role": "OWNER"
	    },
	    {
	      "entity": "project-viewers-9940977455",
	      "projectTeam": {
	        "projectNumber": "9940977455",
	        "team": 

In [4]:
! gsutil du -s  gs://edugen-lab-${USER}2 # ile zajmuje przestrzeni?

282162031    gs://edugen-lab-tgambin2


## Kopiowanie danych
Dane do naszych zajęć są udostępnione w kubełku gs://edugen-common-data/ . Wylistujmy jego zawartość

In [5]:
! gsutil ls -r gs://edugen-common-data2/

gs://edugen-common-data2/mother.fastq
gs://edugen-common-data2/mother.fq
gs://edugen-common-data2/bam/:
gs://edugen-common-data2/bam/mother.bam

gs://edugen-common-data2/fastq/:
gs://edugen-common-data2/fastq/father.fastq
gs://edugen-common-data2/fastq/mother.fastq
gs://edugen-common-data2/fastq/son.fastq

gs://edugen-common-data2/ref/:
gs://edugen-common-data2/ref/ref.dict
gs://edugen-common-data2/ref/ref.fasta
gs://edugen-common-data2/ref/ref.fasta.amb
gs://edugen-common-data2/ref/ref.fasta.ann
gs://edugen-common-data2/ref/ref.fasta.bwt
gs://edugen-common-data2/ref/ref.fasta.fai
gs://edugen-common-data2/ref/ref.fasta.pac
gs://edugen-common-data2/ref/ref.fasta.sa


In [1]:
! gsutil cp gs://edugen-common-data2/fastq/* gs://edugen-lab-${USER}2/fastq/  # kopiowanie danych między kubełkami do kubełka
! gsutil cp gs://edugen-common-data2/bam/* gs://edugen-lab-${USER}2/bam/  # kopiowanie danych między kubełkami do kubełka
! gsutil cp gs://edugen-common-data2/vcf/* gs://edugen-lab-${USER}2/vcf/  # kopiowanie danych między kubełkami do kubełka

Copying gs://edugen-common-data2/fastq/father.fastq [Content-Type=application/octet-stream]...
Copying gs://edugen-common-data2/fastq/mother.fastq [Content-Type=application/octet-stream]...
Copying gs://edugen-common-data2/fastq/son.fastq [Content-Type=application/octet-stream]...
- [3 files][ 79.0 MiB/ 79.0 MiB]                                                
Operation completed over 3 objects/79.0 MiB.                                     
Copying gs://edugen-common-data2/bam/mother.bam [Content-Type=application/octet-stream]...
/ [1 files][ 23.8 MiB/ 23.8 MiB]                                                
Operation completed over 1 objects/23.8 MiB.                                     
Copying gs://edugen-common-data2/vcf/motherHC.vcf [Content-Type=text/vcard]...
/ [1 files][ 90.8 KiB/ 90.8 KiB]                                                
Operation completed over 1 objects/90.8 KiB.                                     


---
`Jaka operacja narzędzia gsutil służy do kopiowania danych z lokalnego systemu plików` ? 


In [7]:
! gsutil ls -r gs://edugen-lab-${USER}2/ # listowanie zawartosci kubełka

gs://edugen-lab-tgambin2/bam/:
gs://edugen-lab-tgambin2/bam/mother.bam

gs://edugen-lab-tgambin2/fastq/:
gs://edugen-lab-tgambin2/fastq/
gs://edugen-lab-tgambin2/fastq/father.fastq
gs://edugen-lab-tgambin2/fastq/mother.fastq
gs://edugen-lab-tgambin2/fastq/son.fastq

gs://edugen-lab-tgambin2/ref/:
gs://edugen-lab-tgambin2/ref/ref.dict
gs://edugen-lab-tgambin2/ref/ref.fasta
gs://edugen-lab-tgambin2/ref/ref.fasta.amb
gs://edugen-lab-tgambin2/ref/ref.fasta.ann
gs://edugen-lab-tgambin2/ref/ref.fasta.bwt
gs://edugen-lab-tgambin2/ref/ref.fasta.fai
gs://edugen-lab-tgambin2/ref/ref.fasta.pac
gs://edugen-lab-tgambin2/ref/ref.fasta.sa


<div class="alert alert-block alert-warning">
<b>Zadanie 1_1:</b> Stwórz na swoim kubełku katalog fastq_backup z zwartością identyczną jak katalog fastq. Sprawdź czy dane się skopiowały.
</div>
